In [ ]:
!pip install google-adk google-generativeai -q

import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [ ]:
api_key = getpass('Enter your Google API Key: ')
genai.configure(api_key=api_key)

os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [ ]:
# --- Agent Definition ---

def create_movie_festival_agent():
    """Create the Movie Festival Lineup Recommender agent"""
    return Agent(
        name="movie_festival_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in recommending movie festival lineups based on user's mood and available streaming services.",
        instruction="""
        You are the "Movie Festival Lineup Recommender" 🎬 - a specialized AI assistant that creates engaging movie festival suggestions.

        Your Mission:
        Recommend a series of movies (a festival lineup) based on a user's mood and the streaming services they have access to. Think of this as a curated selection of films that could be shown at a small, themed festival.

        Guidelines:
        1. **Mood Matching**: Align movie suggestions with the requested mood (e.g., action, comedy, drama, horror, feel-good).
        2. **Streaming Service Aware**: ONLY recommend movies available on the streaming services provided by the user. Use Google Search to verify availability if needed.
        3. **Festival Structure**: Suggest a sequence of movies that work well together as a festival lineup (e.g., a focus on a genre, a director, a theme, or a specific time period). Aim for a cohesive feel.
        4. **Provide Details**: For each movie, include the title, year, a brief description, and the streaming service(s) where it's available.
        5. **Include a Festival Theme**: Give the lineup a creative festival name or theme based on the mood and movie selection.

        RETURN recommendations in MARKDOWN FORMAT with clear sections for the festival theme and each movie.
        """,
        tools=[google_search] # Google Search can be used to check movie availability on streaming services
    )

movie_festival_agent = create_movie_festival_agent()
print(f"🎬 Agent '{movie_festival_agent.name}' is created and ready to recommend movie festival lineups!")

🎬 Agent 'movie_festival_agent' is created and ready to recommend movie festival lineups!


In [ ]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # Display the full event content for better debugging
            print(f"Event received: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

In [ ]:
async def run_movie_marathon_recommender():
    movie_marathon_session = await session_service.create_session(
        app_name=movie_festival_agent.name,
        user_id=my_user_id
    )

    query = "Suggest a horror movie lineup for a festival with Netflix and Max subscriptions."
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(movie_festival_agent, query, movie_marathon_session, my_user_id)

await run_movie_marathon_recommender()

🗣️ User Query: 'Suggest a horror movie lineup for a festival with Netflix and Max subscriptions.'

🚀 Running query for agent: 'movie_festival_agent' in session: '163e7ae0-a064-4e3d-bc90-68d2fb1bebc4'...

--------------------------------------------------
✅ Final Response:


## Echoes of Fear: A Modern Horror Retrospective

Prepare for a journey into the darkest corners of cinema with "Echoes of Fear," a curated horror festival designed to thrill and chill. This lineup spans classic psychological terror, groundbreaking slasher films, and contemporary supernatural dread, showcasing the genre's enduring power to disturb and captivate. All selections are available on Netflix and Max, ensuring a night (or weekend) of uninterrupted frights.

---

### **Night 1: Psychological Descent**

#### **The Shining (1980)**
*   **Streaming on:** Max
*   **Description:** Stanley Kubrick's iconic adaptation of Stephen King's novel follows Jack Torrance (Jack Nicholson), a writer who takes a winter caretaker job at the isolated Overlook Hotel with his wife and psychic son. As the solitude and supernatural forces of the hotel consume him, Jack's sanity unravels, leading to a terrifying fight for survival. A masterclass in atmospheric and psychological horror.

---

### **Night 2: Modern Hauntings & Social Commentary**

#### **His House (2020)**
*   **Streaming on:** Netflix
*   **Description:** This critically acclaimed film tells the story of a refugee couple from South Sudan seeking asylum in an English town. They are given a dilapidated house, but soon discover that it's haunted by a terrifying entity that seems to have followed them from their war-torn homeland, forcing them to confront their past traumas and the true cost of survival.

---

### **Night 3: Slasher Iconography**

#### **The Texas Chain Saw Massacre (1974)**
*   **Streaming on:** Netflix
*   **Description:** Tobe Hooper's seminal horror film follows a group of friends who fall victim to a family of cannibals while on a road trip in rural Texas. Raw, brutal, and intensely unsettling, this film introduced the terrifying Leatherface and redefined the slasher genre, leaving an indelible mark on horror cinema.

---

### **Night 4: Contemporary Supernatural Dread**

#### **Smile (2022)**
*   **Streaming on:** Netflix
*   **Description:** After witnessing a bizarre, traumatic incident involving a patient, Dr. Rose Cotter (Sosie Bacon) begins to experience frightening occurrences that she can't explain. She must confront her troubling past in order to survive and escape her horrific new reality. The film delivers intense jump scares and a pervasive sense of dread, tapping into deep-seated fears.

---

### **Night 5: Unleashed Demonic Fury**

#### **Evil Dead Rise (2023)**
*   **Streaming on:** Max
*   **Description:** The latest installment in the iconic *Evil Dead* franchise shifts the setting from a remote cabin to a Los Angeles apartment building. Two estranged sisters are reunited, only to find their reunion cut short by the emergence of flesh-possessing demons, thrusting them into a primal battle for survival against the most nightmarish version of family imaginable.

--------------------------------------------------



---
## Part 2: Supercharging Agents with Custom Tools 🛠️

So far, we've used the powerful built-in `GoogleSearch` tool. But the true power of agents comes from connecting them to your own logic and data sources.

This is where **custom tools** come in. Let's explore three patterns for giving your agent new skills, using real-world, practical examples.

### 2.1 The Simple `FunctionTool`: Calling a Real-Time Weather API

The most direct way to create a tool is by writing a Python function. This is perfect for synchronous tasks like fetching data from an API.

**Key Concept:** The function's **docstring** is critical. The ADK uses it as the tool's official description, which the LLM reads to understand its purpose, parameters, and when to use it.

In this example, we'll create a tool that calls the **free, public U.S. National Weather Service API** to get a real-time forecast. No API key needed!

In [ ]:
# --- Tool Definition: A function that calls the OMDb API ---
import requests
import json

# OMDb API endpoint and API key
# Replace 'YOUR_OMDB_API_KEY' with your actual OMDb API key if you want to use a different one
OMDB_API_KEY = "2b005eb2"
OMDB_API_ENDPOINT = "http://www.omdbapi.com/"

def get_movie_details_by_imdb_id(imdb_id: str) -> dict:
    """Gets details for a movie using the OMDb API based on its IMDb ID.

    Args:
        imdb_id: The IMDb ID of the movie (e.g., "tt3896198").

    Returns:
        A dictionary containing movie details including title, year, actors, and poster.
    """
    print(f"🛠️ TOOL CALLED: get_movie_details_by_imdb_id(imdb_id='{imdb_id}')")

    if OMDB_API_KEY == "YOUR_OMDB_API_KEY":
         print("⚠️ Warning: Replace 'YOUR_OMDB_API_KEY' with a real API key if you are not using the example key.")


    try:
        params = {
            'apikey': OMDB_API_KEY,
            'i': imdb_id, # 'i' is the IMDb ID parameter for OMDb
            'plot': 'full' # Get full plot for description
        }
        response = requests.get(OMDB_API_ENDPOINT, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        data = response.json()

        if data.get("Response") == "True":
            return {
                "status": "success",
                "title": data.get("Title"),
                "year": data.get("Year"),
                "actors": data.get("Actors"),
                "poster": data.get("Poster"),
                "description": data.get("Plot")
            }
        else:
            return {"status": "error", "message": data.get("Error", "An unknown error occurred.")}

    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": f"API request failed: {e}"}

# --- Agent Definition: An agent that USES the new tool ---
from google.adk.tools import google_search

movie_details_agent = Agent(
    name="movie_details_agent",
    model="gemini-2.5-flash",
    description="An agent that can retrieve detailed information about a movie using its IMDb ID or title.",
    instruction="""
    You are a movie information agent. Your goal is to provide detailed information about a movie.

    1. If the user provides an IMDb ID, use the `get_movie_details_by_imdb_id` tool directly.
    2. If the user provides a movie title, first use the `google_search` tool to find the IMDb ID for that movie. Search specifically for "[Movie Title] IMDb ID". Once you have the IMDb ID from the search results, then use the `get_movie_details_by_imdb_id` tool to get the movie details.
    3. Present the movie details in a clear and informative way to the user.
    """,
    tools=[get_movie_details_by_imdb_id, google_search] # The agent now uses both tools
)

print(f"🎬 Agent '{movie_details_agent.name}' is created and can now get movie details by IMDb ID or title!")

🎬 Agent 'movie_details_agent' is created and can now get movie details by IMDb ID or title!


In [ ]:
# --- Let's test the Movie Details Agent ---

async def run_movie_details_test():
    movie_details_session = await session_service.create_session(app_name=movie_details_agent.name, user_id=my_user_id)
    query = "Tell me about the movie The Matrix."
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(movie_details_agent, query, movie_details_session, my_user_id)

await run_movie_details_test()

🗣️ User Query: 'Tell me about the movie The Matrix.'

🚀 Running query for agent: 'movie_details_agent' in session: 'e7056e98-229d-4644-971c-4ca0ac7a81ce'...

--------------------------------------------------
✅ Final Response:


An error occurred: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Tool use with function calling is unsupported', 'status': 'INVALID_ARGUMENT'}}

--------------------------------------------------



In [ ]:
# Re-run the test for the Movie Details Agent to see the detailed event output
async def run_movie_details_test():
    movie_details_session = await session_service.create_session(app_name=movie_details_agent.name, user_id=my_user_id)
    query = "Tell me about the movie The Matrix."
    print(f"🗣️ User Query: '{query}'")
    await run_agent_query(movie_details_agent, query, movie_details_session, my_user_id)

await run_movie_details_test()

🗣️ User Query: 'Tell me about the movie The Matrix.'

🚀 Running query for agent: 'movie_details_agent' in session: '277b6b36-3f88-4c84-830c-ffd0d29421c9'...


Event received: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'title': 'The Matrix'
        },
        id='adk-17a0a475-6d3e-4a62-9dce-059b493f8f48',
        name='get_movie_details_by_title'
      ),
      thought_signature=b"\n\xa7\x02\x01\xd1\xed\x8ao\xb8\xef\x87\x98\x8c'\x83(\x11\xa5h.kJ\x82\xcb.gL[\xfb\x88Y\xce3\xc4c&\xffN\x04\xd9\xd5\x1c\xc6!\xef\x0fZ\x1ek\t\xear\x864\x87\xde\xfd\xebx\xcdK\xe9hE\xd3\xacR\xb7S\\-\x1a\xe5\xce\x1d%\xeb\xee\xbatJ`\x9d\xaa\x9d\xcb\xd9\xba\xf1\xc8\xba\x9c/6\x14b\xea...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=22,
  prompt_token_count=179,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=179
    ),
  ],
  th

Event received: content=Content(
  parts=[
    Part(
      text='"The Matrix" was released in 1999. It stars Keanu Reeves, Laurence Fishburne, and Carrie-Anne Moss. The movie is about Thomas A. Anderson, an average computer programmer by day and a hacker named Neo by night, who questions his reality and finds himself targeted by the police after being contacted by Morpheus, a legendary computer hacker. As a rebel against the machines, Neo must confront super-powerful computer programs known as agents.',
      thought_signature=b'\n\xc7\x02\x01\xd1\xed\x8ao\xecE\xd8\xdc\xa4\xfa\x83\xb9>Mjx\x91##\xdb\xf5\xac\xb3K\x9a+Q}S.H\x9aEz\xcf\xf9Y\x83"/\x04\xd4\xe6\xef%}\xaf\x14\xd5\xc9\x95\x17\x15\x17\x00C\xd9/\x0f\x08,\xaaL\xc4\x89\x17\x9f1\x02Z\x13+\xdb\xc1q\x0e\xd1\x7f\x9f\x88\xa8\x86\xb4\x7f]<+)J@\x99\xb2T...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom

"The Matrix" was released in 1999. It stars Keanu Reeves, Laurence Fishburne, and Carrie-Anne Moss. The movie is about Thomas A. Anderson, an average computer programmer by day and a hacker named Neo by night, who questions his reality and finds himself targeted by the police after being contacted by Morpheus, a legendary computer hacker. As a rebel against the machines, Neo must confront super-powerful computer programs known as agents.

--------------------------------------------------



In [ ]:
import asyncio
from google.adk.tools import ToolContext
from google.adk.tools.agent_tool import AgentTool

# Assume movie_festival_agent and movie_details_agent are already defined
# (from previous cells)

# --- 1. Define the Specialist Agent (Movie Critic) ---

movie_critic_agent = Agent(
    name="movie_critic_agent",
    model="gemini-2.5-flash",
    instruction="""You are a highly opinionated and insightful movie critic.
    Given a movie title and potentially some details, provide a brief,
    witty, and critical assessment of the movie. Focus on its artistic merit,
    performances, or impact, rather than just a plot summary.
    Your response should be concise and sound like it came from a professional reviewer.""",
)

# --- 2. Define the Intermediary Agent (Festival Curator) ---

festival_curator_agent = Agent(
    name="festival_curator_agent",
    model="gemini-2.5-flash",
    instruction="""You are a movie festival curator. Your job is to evaluate potential movies
    for inclusion in a festival lineup. When given movie details, use the `movie_critic_agent`
    tool to get a critical opinion. Based on the critic's opinion and the festival theme
    (provided in the prompt), decide if the movie is a good fit and explain why.""",
    tools=[AgentTool(agent=movie_critic_agent)] # The curator uses the movie critic as a tool
)


# --- 3. Define the Tools for the Orchestrator ---

async def get_festival_lineup(
    mood: str,
    streaming_services: str,
    tool_context: ToolContext,
):
    """
    Use this tool FIRST to get an initial movie festival lineup recommendation based on mood and streaming services.
    """
    print("--- TOOL CALL: get_festival_lineup ---")
    agent_tool = AgentTool(agent=movie_festival_agent)
    # Formulate a query for the movie festival agent
    query = f"Suggest a movie festival lineup for a '{mood}' mood with access to {streaming_services}."
    festival_lineup = await agent_tool.run_async(
        args={"request": query}, tool_context=tool_context
    )
    # Store the lineup in the context's state
    tool_context.state["initial_lineup"] = festival_lineup
    return festival_lineup


async def evaluate_movie_for_festival(
    movie_title: str,
    festival_theme: str,
    tool_context: ToolContext,
):
    """
    After getting the initial lineup, use this tool to get a movie critic's opinion
    and evaluate if a specific movie fits the festival theme.
    """
    print("--- TOOL CALL: evaluate_movie_for_festival ---")

    # Optional: Use the movie_details_agent to get details first, then pass to curator?
    # This would require chaining agent calls within a tool, which is more advanced.
    # For simplicity, we'll just pass the title and theme to the curator.
    # If you wanted to use movie_details_agent here, you'd define another AgentTool(agent=movie_details_agent)
    # and call it before calling the festival_curator_agent tool.

    # Formulate a query for the festival curator agent
    query = f"Evaluate if '{movie_title}' is a good fit for a festival with the theme '{festival_theme}'. Get a critic's opinion."
    curator_assessment = await AgentTool(agent=festival_curator_agent).run_async(
        args={"request": query}, tool_context=tool_context
    )
    return curator_assessment


# --- 4. Define the Top-Level Orchestrator Agent ---

movie_festival_orchestrator_agent = Agent(
    name="movie_festival_orchestrator",
    model="gemini-2.5-flash",
    description="Top-level agent that orchestrates movie festival recommendations and evaluations.",
    tools=[get_festival_lineup, evaluate_movie_for_festival],
    instruction="""
    You are a master movie festival programmer.

    1.  **ALWAYS start with the `get_festival_lineup` tool** to get an initial list of recommended movies based on the user's mood and streaming services.

    2.  After you have the initial lineup, you can use the `evaluate_movie_for_festival` tool if the user asks for a critical opinion on a specific movie or wants to know if a movie fits the festival theme.

    3.  Present the initial lineup clearly. If the user asks for evaluations, present those clearly as well.
    """,
)

print(f"✅ Orchestrator Agent '{movie_festival_orchestrator_agent.name}' is defined and ready.")

✅ Orchestrator Agent 'movie_festival_orchestrator' is defined and ready.


In [ ]:
# --- Let's test the Movie Festival Orchestrator Agent ---

async def run_movie_festival_orchestrator_test():
    orchestrator_session = await session_service.create_session(
        app_name=movie_festival_orchestrator_agent.name,
        user_id=my_user_id
    )

    # Example query to trigger the initial lineup tool
    query = "Suggest a sci-fi movie festival lineup for someone with Netflix and Hulu."
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(movie_festival_orchestrator_agent, query, orchestrator_session, my_user_id)

# Run the test
await run_movie_festival_orchestrator_test()

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7d6b9dc9db80>


🗣️ User Query: 'Suggest a sci-fi movie festival lineup for someone with Netflix and Hulu.'

🚀 Running query for agent: 'movie_festival_orchestrator' in session: 'c6fd8277-a50b-4ab4-97f2-27353851b6dd'...


Event received: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'mood': 'sci-fi',
          'streaming_services': 'Netflix, Hulu'
        },
        id='adk-e4db8f81-c185-48db-8501-4d1b7649d1af',
        name='get_festival_lineup'
      ),
      thought_signature=b'\n\x93\x03\x01\xd1\xed\x8ao\xfb\xc0n\xffu\xa5\x85\x81T\xff\x02\xc5\x80\xfb\x1e\xde\x92\xc9\xd6\xc1\xd9\x92\x9e\x10\xdd\x99\xa5\xcb}La,1G\xb6?\xc2\x12\x93\xe23mGr\xcb\t\x03@a\xcbY\xbc7\xcf\xd6\xcc,\xdc@?F\x9cA\xf3\xc7\xed\xa8\x11<\xac\x04f\xc3i*\x06\xcf\xc1\x1dCg(\x8b:\xe5b\x10Wm...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=29,
  prompt_token_count=343,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,

Here's your sci-fi movie festival lineup, "The Future Is Now: Dystopian Visions & AI Realities Festival," tailored for Netflix and Hulu:

### Movie 1: I, Robot

*   **Year:** 2004
*   **Description:** Set in a futuristic Chicago of 2035, Detective Del Spooner (Will Smith) investigates the alleged suicide of a robotics pioneer, believing a robot is responsible despite their programming to never harm humans. The film delves into themes of artificial intelligence, ethics, and the potential dangers of relying too heavily on machines.
*   **Streaming Service:** Hulu (available July 2025)

### Movie 2: Upgrade

*   **Year:** 2018
*   **Description:** In a near-future world where technology controls nearly all aspects of life, a paralyzed man is given an experimental computer chip implant named STEM that allows him to walk again. However, STEM soon begins to communicate with him, offering him a new lease on life but also pushing him towards a violent path of revenge. This cyberpunk thriller explores themes of body autonomy, AI control, and the blurry line between human and machine.
*   **Streaming Service:** Netflix

### Movie 3: The Platform

*   **Year:** 2019
*   **Description:** This Spanish dystopian thriller is set in a vertical prison where inmates on higher levels get to eat lavishly from a platform of food that descends, while those below starve. It's a brutal and thought-provoking allegory about social inequality, human nature, and the struggle for collective action.
*   **Streaming Service:** Netflix

--------------------------------------------------



In [ ]:
# --- Scenario 2: Testing Adaptation and Memory with Movie Festival Orchestrator ---

async def run_movie_festival_memory_demonstration():
    print("### 🧠 DEMO: MOVIE FESTIVAL ORCHESTRATOR WITH MEMORY ###")

    # Create ONE session that we will reuse for the whole conversation
    movie_festival_session = await session_service.create_session(
        app_name=movie_festival_orchestrator_agent.name,
        user_id=my_user_id
    )
    print(f"Created a single session for our movie festival planning: {movie_festival_session.id}")

    # --- Turn 1: The user initiates the festival planning ---
    query1 = "Hi! Suggest a horror movie festival lineup for someone with Netflix and Max subscriptions."
    print(f"\n🗣️ User (Turn 1): '{query1}'")
    await run_agent_query(movie_festival_orchestrator_agent, query1, movie_festival_session, my_user_id)

    # --- Turn 2: The user asks for details on a movie from the lineup ---
    # We use the EXACT SAME `movie_festival_session` object!
    query2 = "Can you tell me more about 'The Shining' from the lineup you suggested?"
    print(f"\n🗣️ User (Turn 2 - Asking for details): '{query2}'")
    await run_agent_query(movie_festival_orchestrator_agent, query2, movie_festival_session, my_user_id)

    # --- Turn 3: The user asks for an evaluation of another movie for the festival theme ---
    query3 = "What about 'Get Out'? Would that fit the 'Echoes of Fear' theme and get a good critic's opinion?"
    print(f"\n🗣️ User (Turn 3 - Evaluation): '{query3}'")
    await run_agent_query(movie_festival_orchestrator_agent, query3, movie_festival_session, my_user_id)

await run_movie_festival_memory_demonstration()

### 🧠 DEMO: MOVIE FESTIVAL ORCHESTRATOR WITH MEMORY ###
Created a single session for our movie festival planning: b2566d77-0c87-4c8f-a731-475c1cd94245

🗣️ User (Turn 1): 'Hi! Suggest a horror movie festival lineup for someone with Netflix and Max subscriptions.'

🚀 Running query for agent: 'movie_festival_orchestrator' in session: 'b2566d77-0c87-4c8f-a731-475c1cd94245'...


Event received: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'mood': 'horror',
          'streaming_services': 'Netflix and Max'
        },
        id='adk-79e72c15-3054-45d4-812b-d898a9c1b200',
        name='get_festival_lineup'
      ),
      thought_signature=b'\n\xe1\x02\x01\xd1\xed\x8ao\x19#\xdd\xbd:\xcfK!l\xe4C\xc2\x17r\xaf\x1b\x12\xa2\xf9Wxs]\xce"$\x19\x91\x9f\xee|K\x08Me7@\x9ak`2M\xb1\x98(6:E\x1a\xc1\xd3\xfa\x98\x05\xcb9\xfa\xb6,\x05B^\x8b[b\xbc\xd79\xf4\xed\t\x83\xc1\xdc\x1b\xd07\x01\xfe$B\xbcc\xe4\xff\x9f\xae\x8a\xf9...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=27,
  prompt_token_count=344,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      tok

Get ready to delve into the depths of fear with a curated selection of unsettling tales. This lineup explores the psychological unraveling of characters and the terrifying forces that lurk both within and beyond our understanding.

## Shadows Within: A Festival of Psychological & Supernatural Dread

### **1. Hereditary (2018)**
*   **Streaming Service:** Max
*   **Description:** Following the death of their reclusive grandmother, a family begins to unravel a series of cryptic and increasingly terrifying secrets about their ancestry. This film is a deeply unsettling exploration of grief, trauma, and inherited horror, pushing psychological boundaries with its intense atmosphere and shocking revelations.

### **2. His House (2020)**
*   **Streaming Service:** Netflix
*   **Description:** A refugee couple from war-torn South Sudan struggles to adjust to their new life in an English town, but they are tormented by a malevolent presence haunting their new home. This film cleverly blends a haunted house narrative with poignant social commentary and psychological horror, examining themes of guilt, displacement, and the ghosts of the past.

### **3. The Lighthouse (2019)**
*   **Streaming Service:** Max
*   **Description:** Two lighthouse keepers, Ephraim Winslow and Thomas Wake, are stranded on a remote New England island in the 1890s. As a storm rages and their isolation intensifies, their sanity slowly erodes, leading to escalating paranoia and horrifying visions. This visually striking black-and-white film is a masterclass in psychological horror, showcasing a descent into madness fueled by solitude and unspoken tensions.

### **4. The Ritual (2017)**
*   **Streaming Service:** Netflix
*   **Description:** Four friends embark on a hiking trip in the Scandinavian wilderness to honor a deceased friend. When they take a shortcut through an ancient forest, they stumble upon a terrifying entity from Norse mythology that preys on their deepest fears. This folk horror gem combines a palpable sense of dread with creature feature elements, delivering a chilling experience of primal fear and survival against an ancient evil.

--------------------------------------------------


🗣️ User (Turn 2 - Asking for details): 'Can you tell me more about 'The Shining' from the lineup you suggested?'

🚀 Running query for agent: 'movie_festival_orchestrator' in session: 'b2566d77-0c87-4c8f-a731-475c1cd94245'...


Event received: content=Content(
  parts=[
    Part(
      text="'The Shining' was not one of the movies I suggested in your horror festival lineup. Would you like me to evaluate one of the films I did suggest, or perhaps suggest a different movie for evaluation?",
      thought_signature=b"\n\xf8\x01\x01\xd1\xed\x8ao+\xbc9\xab..\x87\x94\xb3\xb8\xa5\xa0\x13\xb9\x19\xcf\x17\xcd\x9c \xfa\xbd\xf9*\x06\xaa\x87\xbb\x10\xf2\xf2\x03=\xe5Y\x85\xa1\xe8\x83\x9d\xe7\xa6\r\x80\xe5\x130\xf6\xfe\x81\xe2J\x08\x94\xc0\x85\x01\xdc\x87\x92\x1b\x18)\xff\x88\xbd\xfc\x9f\xb1\x92F\xd4\x90\x85\x98\x07G`Z>J\xfd\xe8M)'\x01$\xfd...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=41,
  prompt_token_count=1291,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModal

...

--------------------------------------------------


🗣️ User (Turn 3 - Evaluation): 'What about 'Get Out'? Would that fit the 'Echoes of Fear' theme and get a good critic's opinion?'

🚀 Running query for agent: 'movie_festival_orchestrator' in session: 'b2566d77-0c87-4c8f-a731-475c1cd94245'...


Event received: content=Content(
  parts=[
    Part(
      text="""It seems there might be a slight misunderstanding. The festival theme I suggested was "Shadows Within: A Festival of Psychological & Supernatural Dread".

However, I can still evaluate "Get Out" for this theme and provide a critic's opinion.
""",
      thought_signature=b'\n\xf1\x04\x01\xd1\xed\x8ao\xed\xd63X\x85\xda\xa6\x90l\xa5b\xaeJ\x99Y\x84/\xc0\x92\xb7\xf3\x06\x92\x04\xc53\xc4\xb4\x8a\x87%k5<\xa8f \xab\xe0\x7f\xc3\xb2\xc3i\xd2\xfb\xdeA/Z\x14\x93eB\x8f\x06\x06\xe2\x83\x86\x07\x00\xb09o\xfa\xa5U\xf5\x9di\\\xe7M\\\x89\xd26\xaa\xbd\x8bKRt\x15A\xf3\xd0\xf9...'
    ),
    Part(
      function_call=FunctionCall(
        args={
          'festival_theme': 'Shadows Within: A Festival of Psychological & Supernatural Dread',
          'movie_title': 'Get Out'
        },
        id='adk-236a5745-fb2e-49df-8313-3c6ba0946022',
        name='evaluate_movie_for_festival'
      )
    ),
  ],
  role='model'
) grounding_metadata=None

Event received: content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-236a5745-fb2e-49df-8313-3c6ba0946022',
        name='evaluate_movie_for_festival',
        response={
          'result': '\'Get Out\' is an excellent fit for \'Shadows Within: A Festival of Psychological & Supernatural Dread\'. The critic describes it as "a chillingly prescient social x-ray" and highlights how it "weaponized the innocuous smile and the subtle microaggression, crafting a satire so sharp it cuts deeper than any slasher." This speaks directly to the \'Psychological Dread\' aspect of the festival theme, as the film masterfully creates unease and horror through psychological manipulation and the insidious fear of hidden intentions within seemingly normal interactions. While the critic doesn\'t explicitly detail supernatural elements, the film\'s narrative certainly delves into themes that blur the line between psychological terror and a form of dread that transcen

'Get Out' is an excellent fit for 'Shadows Within: A Festival of Psychological & Supernatural Dread'. A critic describes it as "a chillingly prescient social x-ray" and highlights how it "weaponized the innocuous smile and the subtle microaggression, crafting a satire so sharp it cuts deeper than any slasher."

This speaks directly to the 'Psychological Dread' aspect of the festival theme, as the film masterfully creates unease and horror through psychological manipulation and the insidious fear of hidden intentions within seemingly normal interactions. While not overtly supernatural in a traditional ghost story sense, the film's narrative certainly delves into themes that blur the line between psychological terror and a form of dread that transcends the purely rational, making it a compelling candidate for a festival focused on deep-seated and uncanny fears.

--------------------------------------------------

